<a href="https://colab.research.google.com/github/eduseiti/ia368v_dd_class_05/blob/main/test_LM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Causal Language Model (CLM) test

Check if the fine-tuned **facebook/opt-125m** model got any better on the simple task of Portuguese text generation, after seeing the mc4 pt dataset samples on the `LM_training.ipynb` notebook.

In [1]:
!pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 8.2 MB/s eta 0:00:00


In [2]:
WORKING_FOLDER="drive/MyDrive/unicamp/ia368v_dd/aula_05"

API_KEYS_FILE="/content/drive/MyDrive/unicamp/ia368v_dd/api_keys_20230324.json"

TRAIN_OUTPUT_FOLDER="./trained_model"

NORMALIZED_DATA_BLOCKS_PARTIAL_FILENAME="normalized_samples_block_*"

In [3]:
import os
from google.colab import drive
import json

In [4]:
drive.mount('/content/drive', force_remount=True)
os.chdir(WORKING_FOLDER)

Mounted at /content/drive


In [12]:
# from comet_ml import Experiment

from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, default_data_collator, TrainerCallback, pipeline  

from multiprocessing import Pool

import pickle

import torch

from tqdm.auto import tqdm

import glob

import numpy as np

from typing import TYPE_CHECKING, Any, Callable, Dict, List, Optional, Tuple, Union

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
device

device(type='cpu')

In [8]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [9]:
MODEL_NAME="facebook/opt-125m"

In [10]:
model = AutoModelForCausalLM.from_pretrained("trained_model/checkpoint-24250").to(device)
print('Parameters', model.num_parameters())

Parameters 125239296


In [14]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [15]:
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)
generator("O céu está carregado")

[{'generated_text': 'O céu está carregado de luz, e o sol está escuro'}]

In [16]:
model_original = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)

In [17]:
generator_original = pipeline('text-generation', model=model_original, tokenizer=tokenizer)
generator_original("O céu está carregado")

[{'generated_text': 'O céu está carregado.\nO céu está carregado'}]

Try one phrase completion

In [28]:
generator("O céu está carregado de ", max_length=40)

[{'generated_text': 'O céu está carregado de ervas, e a terra está cheia de água. O céu está cheio de á'}]

In [29]:
generator_original("O céu está carregado de", max_length=40)

[{'generated_text': 'O céu está carregado de uma cidade de uma cidade de uma cidade de uma cidade de uma cidade'}]

Try another one

In [30]:
generator("O pneu do carro está ", max_length=40)

[{'generated_text': 'O pneu do carro está ileso, mas não é um problema. O motorista do carro está com o carro parado. O motorista'}]

In [31]:
generator_original("O pneu do carro está ", max_length=40)

[{'generated_text': 'O pneu do carro está ilegalmente.\nO carro está mais importante.'}]

And one more

In [32]:
generator("O ônibus estava muito ", max_length=40)

[{'generated_text': 'O ônibus estava muito ilegal, mas a Polícia Militar de São Paulo (PMSP) prendeu um suspeito de ter mat'}]

In [33]:
generator_original("O ônibus estava muito ", max_length=40)

[{'generated_text': 'O ônibus estava muito ilegalmente.\nO ônibus estava muito ilegalmente.'}]